# Win Regedit
Notebook para analise do Regedit do Windows.
Em desenvolvimento, porem, com o scan recursivo concluido.

In [1]:
from __future__ import annotations
import winreg


def regedit_scan(
    hkey: str = "HKEY_LOCAL_MACHINE",
    key_path: str = ""
) -> dict:
    result = {}
    if key_path:
        result[key_path] = {}
    with winreg.OpenKey(getattr(winreg, hkey), key_path) as reg_subkey:
        num_subkeys, num_values, _ = winreg.QueryInfoKey(reg_subkey)
        for i in range(num_subkeys):
            subkey_name = winreg.EnumKey(reg_subkey, i)
            subkey_path = f"{key_path}\\{subkey_name}" if key_path else subkey_name
            result[subkey_path] = {}
        for i in range(num_values):
            value_name, value_data, _ = winreg.EnumValue(reg_subkey, i)
            result[key_path][value_name] = value_data
    return result


def scan_recursive(hkey: str) -> dict:
    paths = {}
    queue = [""]
    while True:
        if not queue:
            break
        key_path = queue.pop(0)
        try:
            res = regedit_scan(hkey, key_path)
            if res:
                paths.update(res)
                queue += list(res.keys())
        except BaseException:
            pass
    return dict([(k, paths[k]) for k in sorted(paths.keys())])


In [2]:
hkeys = (
    "HKEY_LOCAL_MACHINE",
    "HKEY_CLASSES_ROOT",
    "HKEY_CURRENT_USER",
    "HKEY_USERS",
    "HKEY_PERFORMANCE_DATA",
    "HKEY_CURRENT_CONFIG",
    "HKEY_DYN_DATA",
)

paths = {}
for hkey in hkeys:
    paths[hkey] = scan_recursive(hkey)

paths